<a href="https://colab.research.google.com/github/skywalker0803r/c620/blob/main/notebook/ETL_C620.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from tqdm import tqdm_notebook as tqdm
def get_col(df,name):
    return df.columns[df.columns.str.contains(name)].tolist()

In [2]:
path = '/content/drive/MyDrive/台塑輕油案子/data/c620/raw/c620/Aroma-2 C620_Split Factors Calculation for AI Training_Overall_Data.xlsx'

In [3]:
xl = pd.ExcelFile(path)

In [4]:
def clean_data(df):
    df = df.T.reset_index()
    idx = df.iloc[6:,1].values
    df = df.iloc[:,2:]
    prefix = pd.Series(df.iloc[1,:].values).fillna(method='ffill').values
    middle = pd.Series(df.iloc[2,:].values).fillna(method='ffill').values
    middle_2 = pd.Series(df.iloc[3,:].values).fillna(method='ffill').values
    suffix = pd.Series(df.iloc[4,:].values).fillna(method='ffill').values
    col = [p.strip() + '_' + m1.strip() + '_' + m2.strip() + '_' + s.strip() for p,m1,m2,s in zip(prefix,middle,middle_2,suffix)]
    df = df.iloc[6:,:]
    df.index = idx
    df.columns = col
    return df.astype('float')

In [5]:
train = pd.DataFrame()
for i in tqdm(xl.sheet_names):
    rows = clean_data(xl.parse(i))
    if len(train) == 0:
      train = rows
    else:
      train = train.append(rows)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


In [6]:
train.head()

,Combined Feed to Tatoray Stripper C620_Flow Rate_Volume Flow Rate_m3/hr,Combined Feed to Tatoray Stripper C620_Flow Rate_Mass Flow Rate_ton/hr,Combined Feed to Tatoray Stripper C620_Detailed Composition_Hydrogen_wt%,Combined Feed to Tatoray Stripper C620_Detailed Composition_Methane_wt%,Combined Feed to Tatoray Stripper C620_Detailed Composition_Ethane_wt%,Combined Feed to Tatoray Stripper C620_Detailed Composition_Propane_wt%,Combined Feed to Tatoray Stripper C620_Detailed Composition_n-Butane_wt%,Combined Feed to Tatoray Stripper C620_Detailed Composition_n-Pentane_wt%,Combined Feed to Tatoray Stripper C620_Detailed Composition_n-Hexane_wt%,Combined Feed to Tatoray Stripper C620_Detailed Composition_Benzene_wt%,Combined Feed to Tatoray Stripper C620_Detailed Composition_Cyclohexane_wt%,Combined Feed to Tatoray Stripper C620_Detailed Composition_n-Heptane_wt%,Combined Feed to Tatoray Stripper C620_Detailed Composition_Water_wt%,Combined Feed to Tatoray Stripper C620_Detailed Composition_Methylcyclohexane_wt%,Combined Feed to Tatoray Stripper C620_Detailed Composition_Toluene_wt%,Combined Feed to Tatoray Stripper C620_Detailed Composition_n-Octane_wt%,Combined Feed to Tatoray Stripper C620_Detailed Composition_n-Propylcyclopentane_wt%,Combined Feed to Tatoray Stripper C620_Detailed Composition_Ethylcyclohexane_wt%,Combined Feed to Tatoray Stripper C620_Detailed Composition_Ethylbenzene_wt%,Combined Feed to Tatoray Stripper C620_Detailed Composition_p-Xylene_wt%,Combined Feed to Tatoray Stripper C620_Detailed Composition_m-Xylene_wt%,Combined Feed to Tatoray Stripper C620_Detailed Composition_o-Xylene_wt%,Combined Feed to Tatoray Stripper C620_Detailed Composition_n-Nonane_wt%,Combined Feed to Tatoray Stripper C620_Detailed Composition_i-Propylbenzene_wt%,Combined Feed to Tatoray Stripper C620_Detailed Composition_n-Propylcyclohexane_wt%,Combined Feed to Tatoray Stripper C620_Detailed Composition_n-Propylbenzene_wt%,Combined Feed to Tatoray Stripper C620_Detailed Composition_1-Methyl-3-ethylbenzene_wt%,Combined Feed to Tatoray Stripper C620_Detailed Composition_1-Methyl-4-ethylbenzene_wt%,"Combined Feed to Tatoray Stripper C620_Detailed Composition_1,3,5-Trimethylbenzene_wt%",Combined Feed to Tatoray Stripper C620_Detailed Composition_1-Methyl-2-ethylbenzene_wt%,"Combined Feed to Tatoray Stripper C620_Detailed Composition_1,2,4-Trimethylbenzene_wt%",Combined Feed to Tatoray Stripper C620_Detailed Composition_tert-Butylcyclohexane_wt%,"Combined Feed to Tatoray Stripper C620_Detailed Composition_1,2,3-Trimethylbenzene_wt%",Combined Feed to Tatoray Stripper C620_Detailed Composition_Indane_wt%,Combined Feed to Tatoray Stripper C620_Detailed Composition_1-Methyl-4-n-propylbenzene_wt%,"Combined Feed to Tatoray Stripper C620_Detailed Composition_1,2-Diethylbenzene_wt%",Combined Feed to Tatoray Stripper C620_Detailed Composition_5-Ethyl-m-xylene_wt%,"Combined Feed to Tatoray Stripper C620_Detailed Composition_1,4-Diethylbenzene_wt%","Combined Feed to Tatoray Stripper C620_Detailed Composition_1,2,3,5-Tetramethylbenzene_wt%",Combined Feed to Tatoray Stripper C620_Detailed Composition_n-Pentylbenzene_wt%,...,Tatoray Stripper C620 Split Factors Calculation_Split Factor for Individual Component to Tatoray Stripper C620 Bottoms_Methane_Fraction,Tatoray Stripper C620 Split Factors Calculation_Split Factor for Individual Component to Tatoray Stripper C620 Bottoms_Ethane_Fraction,Tatoray Stripper C620 Split Factors Calculation_Split Factor for Individual Component to Tatoray Stripper C620 Bottoms_Propane_Fraction,Tatoray Stripper C620 Split Factors Calculation_Split Factor for Individual Component to Tatoray Stripper C620 Bottoms_n-Butane_Fraction,Tatoray Stripper C620 Split Factors Calculation_Split Factor for Individual Component to Tatoray Stripper C620 Bottoms_n-Pentane_Fraction,Tatoray Stripper C620 Split Factors Calculation_Split Factor for Individual Component to Tatoray Stripper C620 Bottoms_n-Hexane_Fraction,Tatoray Stripper C620 Sp

In [7]:
# Function to calculate missing values by column# Funct 
def missing_values_table(df):
        # Total missing values
        mis_val = df.isnull().sum()
        
        # Percentage of missing values
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        
        # Make a table with the results
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        
        # Rename the columns
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        
        # Sort the table by percentage of missing descending
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        
        # Print some summary information
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        
        # Return the dataframe with missing information
        return mis_val_table_ren_columns
# Missing values statistics
missing_values = missing_values_table(train)
missing_values.head(20)

Your selected dataframe has 420 columns.
There are 5 columns that have missing values.


,Missing Values,% of Total Values
Tatoray Stripper C620 Operation_Bottoms Production Rate and Composition_Sum_wt%,15552,100.0
Tatoray Stripper C620 Split Factors Calculation_Split Factor for Individual Component to Tatoray Stripper C620 Vent Gas_Vent Gas Split Factor_wt%,15552,100.0
Tatoray Stripper C620 Split Factors Calculation_Split Factor for Individual Component to Tatoray Stripper C620 Distillate_Dist Split Factor_Fraction,15552,100.0
Tatoray Stripper C620 Split Factors Calculation_Split Factor for Individual Component to Tatoray Stripper C620 Sidedraw_Sidedraw Split Factor_Fraction,15552,100.0
Tatoray Stripper C620 Split Factors Calculation_Split Factor for Individual Component to Tatoray Stripper C620 Bottoms_Btms Split Factor_Fraction,15552,100.0


In [8]:
print(train.shape)
train = train.dropna(axis=1)
train.shape

(15552, 420)


(15552, 415)

In [9]:
fr = get_col(train,'Mass Flow Rate')
train[fr]

,Combined Feed to Tatoray Stripper C620_Flow Rate_Mass Flow Rate_ton/hr,Tatoray Stripper C620 Operation_Feed Properties_Mass Flow Rate_ton/hr,Tatoray Stripper C620 Operation_Vent Gas Production Rate and Composition_Mass Flow Rate_ton/hr,Tatoray Stripper C620 Operation_Distillate Production Rate and Composition_Mass Flow Rate_ton/hr,Tatoray Stripper C620 Operation_Sidedraw Production Rate and Composition_Mass Flow Rate_ton/hr,Tatoray Stripper C620 Operation_Bottoms Production Rate and Composition_Mass Flow Rate_ton/hr
001-001,153.483047,153.483047,2.070491,0.392216,26.038795,124.981539
001-002,153.483047,153.483047,2.070494,0.392216,26.038781,124.981555
001-003,153.483047,153.483047,2.070492,0.392216,26.038785,124.981555
001-004,153.483047,153.483047,2.007272,0.790220,25.672561,125.012984
001-005,153.483047,153.483047,2.007274,0.790220,25.672559,125.012984
...,...,...,...,...,...,...
144-104,185.875844,185.875844,2.108982,0.399545,35.107414,148.259922
144-105,185.875844,185.875844,2.108984,0.399545,35.107430,148.259891
144-106,185.875844,185.875844,2.101170,0.399745,40.144824,143.230125
144-107,185.875844,185.875844,2.101170,0.399745,40.144824,143.230125


In [10]:
fr.remove(fr[1])
check_df = train[fr]
check_df.to_csv('/content/drive/MyDrive/台塑輕油案子/data/c620/cleaned/{}_mbcheck.csv'.format('c620'))

In [12]:
x41 = get_col(train,'Detailed Composition')[:-1]
case = get_col(train,'Specifications')[:-1]

yRefluxRate = get_col(train, 'Reflux Rate')
yControl = get_col(train, 'Control')
yHeatDuty = get_col(train, 'Heat Duty_Mkcal')

vent_gas_x = get_col(train,'Operation_Vent Gas')[2:-1]
distillate_x = get_col(train,'Operation_Distillate')[2:-1]
sidedraw_x = get_col(train,'Operation_Sidedraw')[2:-1]
bottoms_x = get_col(train,'Operation_Bottoms')[2:-1]

vent_gas_sf = get_col(train,'{} Vent Gas'.format('C620'))
distillate_sf = get_col(train,'{} Distillate'.format('C620'))
sidedraw_sf = get_col(train,'{} Sidedraw'.format('C620'))
bottoms_sf = get_col(train,'{} Bottoms'.format('C620'))

In [13]:
import numpy as np
def sp2wt(x,s):
  a = 100*x*s
  b = np.diag(x@s.T).reshape(-1,1)
  b = np.clip(b,1e-8,np.inf)
  return a/b

In [14]:
for sf,wt in tqdm(zip([vent_gas_sf,distillate_sf,sidedraw_sf,bottoms_sf],[vent_gas_x,distillate_x,sidedraw_x,bottoms_x])):
  assert np.allclose(sp2wt(train[x41].values,train[sf].values),train[wt].values)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


In [15]:
col_names = {}
col_names['x41'] = x41
col_names['case'] = case

col_names['yRefluxRate'] = yRefluxRate
col_names['yControl'] = yControl
col_names['yHeatDuty'] = yHeatDuty

col_names['vent_gas_x'] = vent_gas_x
col_names['distillate_x'] = distillate_x
col_names['sidedraw_x'] = sidedraw_x
col_names['bottoms_x'] = bottoms_x

col_names['vent_gas_sf'] = vent_gas_sf
col_names['distillate_sf'] = distillate_sf
col_names['sidedraw_sf'] = sidedraw_sf
col_names['bottoms_sf'] = bottoms_sf

In [16]:
MFR = get_col(train, 'Mass Flow Rate')
MFR.remove(MFR[0])
VFR = get_col(train, 'Volume Flow Rate')
VFR.remove(VFR[0])

vMFR = train[MFR].values
vVFR = train[VFR].values
n = [ i.split('_')[1] for i in MFR]

col_names['density'] = ['Density_{}'.format(i) for i in n]
density = vMFR/vVFR
density = pd.DataFrame(density,
                       index = train.index,
                       columns = col_names['density'])
density.head()

,Density_Feed Properties,Density_Vent Gas Production Rate and Composition,Density_Distillate Production Rate and Composition,Density_Sidedraw Production Rate and Composition,Density_Bottoms Production Rate and Composition
001-001,0.862264,0.449632,0.784431,0.881339,0.871859
001-002,0.862264,0.449633,0.784431,0.881339,0.871859
001-003,0.862264,0.449633,0.784431,0.881339,0.871859
001-004,0.862264,0.447516,0.790220,0.881372,0.871859
001-005,0.862264,0.447516,0.790220,0.881372,0.871859


In [17]:
col_names.keys()

dict_keys(['x41', 'case', 'yRefluxRate', 'yControl', 'yHeatDuty', 'vent_gas_x', 'distillate_x', 'sidedraw_x', 'bottoms_x', 'vent_gas_sf', 'distillate_sf', 'sidedraw_sf', 'bottoms_sf', 'density'])

In [18]:
print(train.shape)
train = train.join(density)
print(train.shape)

(15552, 415)
(15552, 420)


In [19]:
for k,v in tqdm(col_names.items()):
  assert len(train[v]) != 0

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


In [20]:
train.head()

,Combined Feed to Tatoray Stripper C620_Flow Rate_Volume Flow Rate_m3/hr,Combined Feed to Tatoray Stripper C620_Flow Rate_Mass Flow Rate_ton/hr,Combined Feed to Tatoray Stripper C620_Detailed Composition_Hydrogen_wt%,Combined Feed to Tatoray Stripper C620_Detailed Composition_Methane_wt%,Combined Feed to Tatoray Stripper C620_Detailed Composition_Ethane_wt%,Combined Feed to Tatoray Stripper C620_Detailed Composition_Propane_wt%,Combined Feed to Tatoray Stripper C620_Detailed Composition_n-Butane_wt%,Combined Feed to Tatoray Stripper C620_Detailed Composition_n-Pentane_wt%,Combined Feed to Tatoray Stripper C620_Detailed Composition_n-Hexane_wt%,Combined Feed to Tatoray Stripper C620_Detailed Composition_Benzene_wt%,Combined Feed to Tatoray Stripper C620_Detailed Composition_Cyclohexane_wt%,Combined Feed to Tatoray Stripper C620_Detailed Composition_n-Heptane_wt%,Combined Feed to Tatoray Stripper C620_Detailed Composition_Water_wt%,Combined Feed to Tatoray Stripper C620_Detailed Composition_Methylcyclohexane_wt%,Combined Feed to Tatoray Stripper C620_Detailed Composition_Toluene_wt%,Combined Feed to Tatoray Stripper C620_Detailed Composition_n-Octane_wt%,Combined Feed to Tatoray Stripper C620_Detailed Composition_n-Propylcyclopentane_wt%,Combined Feed to Tatoray Stripper C620_Detailed Composition_Ethylcyclohexane_wt%,Combined Feed to Tatoray Stripper C620_Detailed Composition_Ethylbenzene_wt%,Combined Feed to Tatoray Stripper C620_Detailed Composition_p-Xylene_wt%,Combined Feed to Tatoray Stripper C620_Detailed Composition_m-Xylene_wt%,Combined Feed to Tatoray Stripper C620_Detailed Composition_o-Xylene_wt%,Combined Feed to Tatoray Stripper C620_Detailed Composition_n-Nonane_wt%,Combined Feed to Tatoray Stripper C620_Detailed Composition_i-Propylbenzene_wt%,Combined Feed to Tatoray Stripper C620_Detailed Composition_n-Propylcyclohexane_wt%,Combined Feed to Tatoray Stripper C620_Detailed Composition_n-Propylbenzene_wt%,Combined Feed to Tatoray Stripper C620_Detailed Composition_1-Methyl-3-ethylbenzene_wt%,Combined Feed to Tatoray Stripper C620_Detailed Composition_1-Methyl-4-ethylbenzene_wt%,"Combined Feed to Tatoray Stripper C620_Detailed Composition_1,3,5-Trimethylbenzene_wt%",Combined Feed to Tatoray Stripper C620_Detailed Composition_1-Methyl-2-ethylbenzene_wt%,"Combined Feed to Tatoray Stripper C620_Detailed Composition_1,2,4-Trimethylbenzene_wt%",Combined Feed to Tatoray Stripper C620_Detailed Composition_tert-Butylcyclohexane_wt%,"Combined Feed to Tatoray Stripper C620_Detailed Composition_1,2,3-Trimethylbenzene_wt%",Combined Feed to Tatoray Stripper C620_Detailed Composition_Indane_wt%,Combined Feed to Tatoray Stripper C620_Detailed Composition_1-Methyl-4-n-propylbenzene_wt%,"Combined Feed to Tatoray Stripper C620_Detailed Composition_1,2-Diethylbenzene_wt%",Combined Feed to Tatoray Stripper C620_Detailed Composition_5-Ethyl-m-xylene_wt%,"Combined Feed to Tatoray Stripper C620_Detailed Composition_1,4-Diethylbenzene_wt%","Combined Feed to Tatoray Stripper C620_Detailed Composition_1,2,3,5-Tetramethylbenzene_wt%",Combined Feed to Tatoray Stripper C620_Detailed Composition_n-Pentylbenzene_wt%,...,Tatoray Stripper C620 Split Factors Calculation_Split Factor for Individual Component to Tatoray Stripper C620 Bottoms_n-Hexane_Fraction,Tatoray Stripper C620 Split Factors Calculation_Split Factor for Individual Component to Tatoray Stripper C620 Bottoms_Benzene_Fraction,Tatoray Stripper C620 Split Factors Calculation_Split Factor for Individual Component to Tatoray Stripper C620 Bottoms_Cyclohexane_Fraction,Tatoray Stripper C620 Split Factors Calculation_Split Factor for Individual Component to Tatoray Stripper C620 Bottoms_n-Heptane_Fraction,Tatoray Stripper C620 Split Factors Calculation_Split Factor for Individual Component to Tatoray Stripper C620 Bottoms_Water_Fraction,Tatoray Stripper C620 Split Factors Calculation_Split Factor for Individual Component to Tatoray Stripper C620 Bottoms_Methylcyclohexane_Fraction,Tatoray Stri

In [21]:
import joblib
train.to_csv('/content/drive/MyDrive/台塑輕油案子/data/c620/cleaned/{}_train.csv'.format('c620'))
joblib.dump(col_names,'/content/drive/MyDrive/台塑輕油案子/data/c620/col_names/{}_col_names.pkl'.format('c620'))
print('ok')

ok
